In [1]:
import os
from shutil import copyfile

from torchsummary import summary
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torchvision import transforms

from scripts.models import *
from scripts.utility import *
from scripts.dataloader import *
from scripts.callbacks import *
import matplotlib.pyplot as plt

import matplotlib

from scripts import *

# save plots for latex
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [2]:
potsdam_dir = "../potsdam_data/potsdam_cars_corrected"
artificial_dir = "../potsdam_data/cem-v0/v2/training_tightcanvas_graybackground"

In [3]:
# Dynamic Pad. + Gaus Noise + Color Jitter + Horizontal/Vertical Flip
transform1 = transforms.Compose([transforms.ColorJitter(hue=[-0.1, 0.1]),
                            DynamicPad(min_img_dim=(110, 60),
                                       padding_mode="edge"),
                            transforms.RandomCrop(
                            (55, 105), padding_mode="reflect"),
                            transforms.Resize((32, 64)),
                            transforms.RandomHorizontalFlip(p=0.5),
                            transforms.RandomVerticalFlip(p=0.5),
                            transforms.ToTensor(),
                            transforms.Normalize([0.5], [0.5])])
transform2 = transforms.Compose([transforms.ColorJitter(hue=[-0.1, 0.1]),
                            DynamicPad(min_img_dim=(130, 70),
                                       padding_mode="constant", padding_value=125),
                            transforms.RandomRotation(
                            degrees=5, resample=PIL.Image.NEAREST, fill=125),
                            transforms.RandomCrop(
                            (60, 120), padding_mode="reflect"),
                            transforms.Resize((32, 64)),
                            transforms.RandomHorizontalFlip(p=0.5),
                            transforms.RandomVerticalFlip(p=0.5),
                            transforms.ToTensor(),
                            transforms.Normalize([0.5], [0.5]),
                            AddNoise(alpha=0.07)])

checkpoint_path = "experiments/munit_experiments/lightning_logs/version_0/checkpoints/epoch={}.ckpt"
save_dir = "experiments/munit_experiments/version_0/results"
datasetmodule = PostdamCarsDataModule(potsdam_dir,
                      data_dir2=artificial_dir,
                      transform=transform1,
                      transform2=transform2,
                      batch_size=1024)
datasetmodule.setup()
dataloader = datasetmodule.train_dataloader()

real, artificial = next(iter(dataloader))
styles = th.normal(0, 1, (1024, 8))

max_epochs = 1000
checkpoints = [i-1 for i in range(25, max_epochs, 25)] + [max_epochs-1]

numpy_file = os.path.join(save_dir, "fid_scores.npy")
if not os.path.exists(numpy_file):
    fid_train = []
    # main stuff
    with th.no_grad():
        for epoch in tqdm.tqdm(checkpoints, desc="Checkpoints"):
            path = checkpoint_path.format(epoch)
            if os.path.exists(path):
                # load model
                model = MUNIT.load_from_checkpoint(path)
                model.eval()
                model.cuda()


                encoder1 = model.generator.encoder1
                decoder2 = model.generator.decoder2

                contents, _ = encoder1(artificial.cuda())
                fake = decoder2(contents, styles.cuda())

                fid_train.append(fid_score(real, fake, device="cuda:0"))
                print(f"[FAKE] Epoch {epoch}: Train FID is {fid_train[-1]}.")
    np.save(numpy_file, {"epochs": checkpoints, "fid_train": fid_train})
    
# do the best epoch 5 times
data = np.load(numpy_file, allow_pickle=True).item()
fid_train = data.get("fid_train", [])
idx = np.argmin(fid_train)
epoch = checkpoints[idx]
path = checkpoint_path.format(epoch)

# load model
with th.no_grad():
    model = MUNIT.load_from_checkpoint(path)
    model.eval()
    model.cuda()
    encoder1 = model.generator.encoder1
    decoder2 = model.generator.decoder2
    
    fid_multiple = []
    for _ in range(5):
        real, artificial = next(iter(dataloader))
        styles = th.normal(0, 1, (1024, 8))
        contents, _ = encoder1(artificial.cuda())
        fake = decoder2(contents, styles.cuda())
        fid_multiple.append(fid_score(real, fake, device="cuda:0"))
    fid_multiple = np.array(fid_multiple)
    print(f"FID MIN - {fid_multiple.min()} | FID MAX - {fid_multiple.max()} | FID Mean - {fid_multiple.min()}")
    
# PLOT STUFF
plt.figure()
plt.plot(checkpoints[:len(fid_train)], fid_train, marker="*")
plt.xlabel("Epochs", fontsize=18)
plt.ylabel("FID Score", fontsize=18)
# plt.title("Training FID Score", fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "fid_score_train.pgf"))
# close everything, i don't know how important it is
plt.clf()
plt.close()

# PLOT STUFF
plt.figure()
plt.plot(checkpoints[:len(fid_train)], fid_train, marker="*")
plt.xlabel("Epochs", fontsize=18)
plt.ylabel("FID Score", fontsize=18)
# plt.title("Training FID Score", fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "fid_score_train.png"))
# close everything, i don't know how important it is
plt.clf()
plt.close()

AttributeError: 'list' object has no attribute 'min'

In [4]:
fid_multiple = np.array(fid_multiple)
print(f"FID MIN - {fid_multiple.min()} | FID MAX - {fid_multiple.max()} | FID Mean - {fid_multiple.min()}")

FID MIN - 1.6365012737206928 | FID MAX - 1.7204533475421957 | FID Mean - 1.6365012737206928


In [5]:
# Horizontal/Vertical Flip
transform1 = transforms.Compose([transforms.ColorJitter(hue=[-0.1, 0.1]),
                            transforms.Resize((32, 64)),
                            transforms.RandomHorizontalFlip(p=0.5),
                            transforms.RandomVerticalFlip(p=0.5),
                            transforms.ToTensor(),
                            transforms.Normalize([0.5], [0.5])])
transform2 = transforms.Compose([transforms.Resize((32, 64)),
                            transforms.RandomHorizontalFlip(p=0.5),
                            transforms.RandomVerticalFlip(p=0.5),
                            transforms.ToTensor(),
                            transforms.Normalize([0.5], [0.5])])

checkpoint_path = "experiments/munit_experiments/lightning_logs/version_1/checkpoints/epoch={}.ckpt"
save_dir = "experiments/munit_experiments/version_1/results"
datasetmodule = PostdamCarsDataModule(potsdam_dir,
                      data_dir2=artificial_dir,
                      transform=transform1,
                      transform2=transform2,
                      batch_size=1024)
datasetmodule.setup()
dataloader = datasetmodule.train_dataloader()

real, artificial = next(iter(dataloader))
styles = th.normal(0, 1, (1024, 8))

max_epochs = 1000
checkpoints = [i-1 for i in range(25, max_epochs, 25)] + [max_epochs-1]

numpy_file = os.path.join(save_dir, "fid_scores.npy")
if not os.path.exists(numpy_file):
    fid_train = []
    # main stuff
    with th.no_grad():
        for epoch in tqdm.tqdm(checkpoints, desc="Checkpoints"):
            path = checkpoint_path.format(epoch)
            if os.path.exists(path):
                # load model
                model = MUNIT.load_from_checkpoint(path)
                model.eval()
                model.cuda()


                encoder1 = model.generator.encoder1
                decoder2 = model.generator.decoder2

                contents, _ = encoder1(artificial.cuda())
                fake = decoder2(contents, styles.cuda())

                fid_train.append(fid_score(real, fake, device="cuda:0"))
                print(f"[FAKE] Epoch {epoch}: Train FID is {fid_train[-1]}.")
    np.save(numpy_file, {"epochs": checkpoints, "fid_train": fid_train})

# do the best epoch 5 times
data = np.load(numpy_file, allow_pickle=True).item()
fid_train = data.get("fid_train", [])
idx = np.argmin(fid_train)
epoch = checkpoints[idx]
path = checkpoint_path.format(epoch)

# load model
with th.no_grad():
    model = MUNIT.load_from_checkpoint(path)
    model.eval()
    model.cuda()
    encoder1 = model.generator.encoder1
    decoder2 = model.generator.decoder2
    
    fid_multiple = []
    for _ in range(5):
        real, artificial = next(iter(dataloader))
        styles = th.normal(0, 1, (1024, 8))
        contents, _ = encoder1(artificial.cuda())
        fake = decoder2(contents, styles.cuda())
        fid_multiple.append(fid_score(real, fake, device="cuda:0"))
    fid_multiple = np.array(fid_multiple)
    print(f"FID MIN - {fid_multiple.min()} | FID MAX - {fid_multiple.max()} | FID Mean - {fid_multiple.min()}")
    
# PLOT STUFF
plt.figure()
plt.plot(checkpoints[:len(fid_train)], fid_train, marker="*")
plt.xlabel("Epochs", fontsize=18)
plt.ylabel("FID Score", fontsize=18)
# plt.title("Training FID Score", fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "fid_score_train.pgf"))
# close everything, i don't know how important it is
plt.clf()
plt.close()

# PLOT STUFF
plt.figure()
plt.plot(checkpoints[:len(fid_train)], fid_train, marker="*")
plt.xlabel("Epochs", fontsize=18)
plt.ylabel("FID Score", fontsize=18)
# plt.title("Training FID Score", fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "fid_score_train.png"))
# close everything, i don't know how important it is
plt.clf()
plt.close()

FID MIN - 1.947625301850671 | FID MAX - 2.031603158028278 | FID Mean - 1.947625301850671


In [6]:
 # Color Jitter + Gaus Noise + Horizontal/Vertical Flip
transform1 = transforms.Compose([transforms.ColorJitter(hue=[-0.1, 0.1]),
                            transforms.Resize((32, 64)),
                            transforms.RandomHorizontalFlip(p=0.5),
                            transforms.RandomVerticalFlip(p=0.5),
                            transforms.ToTensor(),
                            transforms.Normalize([0.5], [0.5])])
transform2 =  transforms.Compose([transforms.ColorJitter(hue=[-0.1, 0.1]),
                            transforms.Resize((32, 64)),
                            transforms.RandomHorizontalFlip(p=0.5),
                            transforms.RandomVerticalFlip(p=0.5),
                            transforms.ToTensor(),
                            transforms.Normalize([0.5], [0.5]),
                            AddNoise(alpha=0.07)])

checkpoint_path = "experiments/munit_experiments/lightning_logs/version_2/checkpoints/epoch={}.ckpt"
save_dir = "experiments/munit_experiments/version_2/results"
datasetmodule = PostdamCarsDataModule(potsdam_dir,
                      data_dir2=artificial_dir,
                      transform=transform1,
                      transform2=transform2,
                      batch_size=1024)
datasetmodule.setup()
dataloader = datasetmodule.train_dataloader()

real, artificial = next(iter(dataloader))
styles = th.normal(0, 1, (1024, 8))

max_epochs = 1000
checkpoints = [i-1 for i in range(25, max_epochs, 25)] + [max_epochs-1]

numpy_file = os.path.join(save_dir, "fid_scores.npy")
if not os.path.exists(numpy_file):
    fid_train = []
    # main stuff
    with th.no_grad():
        for epoch in tqdm.tqdm(checkpoints, desc="Checkpoints"):
            path = checkpoint_path.format(epoch)
            if os.path.exists(path):
                # load model
                model = MUNIT.load_from_checkpoint(path)
                model.eval()
                model.cuda()


                encoder1 = model.generator.encoder1
                decoder2 = model.generator.decoder2

                contents, _ = encoder1(artificial.cuda())
                fake = decoder2(contents, styles.cuda())

                fid_train.append(fid_score(real, fake, device="cuda:0"))
                print(f"[FAKE] Epoch {epoch}: Train FID is {fid_train[-1]}.")
    np.save(numpy_file, {"epochs": checkpoints, "fid_train": fid_train})

# do the best epoch 5 times
data = np.load(numpy_file, allow_pickle=True).item()
fid_train = data.get("fid_train", [])
idx = np.argmin(fid_train)
epoch = checkpoints[idx]
path = checkpoint_path.format(epoch)

# load model
with th.no_grad():
    model = MUNIT.load_from_checkpoint(path)
    model.eval()
    model.cuda()
    encoder1 = model.generator.encoder1
    decoder2 = model.generator.decoder2
    
    fid_multiple = []
    for _ in range(5):
        real, artificial = next(iter(dataloader))
        styles = th.normal(0, 1, (1024, 8))
        contents, _ = encoder1(artificial.cuda())
        fake = decoder2(contents, styles.cuda())
        fid_multiple.append(fid_score(real, fake, device="cuda:0"))
    fid_multiple = np.array(fid_multiple)
    print(f"FID MIN - {fid_multiple.min()} | FID MAX - {fid_multiple.max()} | FID Mean - {fid_multiple.min()}")

# PLOT STUFF
plt.figure()
plt.plot(checkpoints[:len(fid_train)], fid_train, marker="*")
plt.xlabel("Epochs", fontsize=18)
plt.ylabel("FID Score", fontsize=18)
# plt.title("Training FID Score", fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "fid_score_train.pgf"))
# close everything, i don't know how important it is
plt.clf()
plt.close()

# PLOT STUFF
plt.figure()
plt.plot(checkpoints[:len(fid_train)], fid_train, marker="*")
plt.xlabel("Epochs", fontsize=18)
plt.ylabel("FID Score", fontsize=18)
# plt.title("Training FID Score", fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "fid_score_train.png"))
# close everything, i don't know how important it is
plt.clf()
plt.close()

FID MIN - 1.9946966381925895 | FID MAX - 2.0697958747680576 | FID Mean - 1.9946966381925895


In [7]:
# Dynamic Pad. + Gaus Noise + Color Jitter + Horizontal/Vertical Flip + Larger Image

transform1 = transforms.Compose([transforms.ColorJitter(hue=[-0.1, 0.1]),
                                 DynamicPad(min_img_dim=(110, 60),
                                            padding_mode="edge"),
                                 transforms.RandomCrop(
                                     (55, 105), padding_mode="reflect"),
                                 transforms.Resize((40, 80)),
                                 transforms.RandomHorizontalFlip(p=0.5),
                                 transforms.RandomVerticalFlip(p=0.5),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.5], [0.5])])

transform2 = transforms.Compose([transforms.ColorJitter(hue=[-0.1, 0.1]),
                                 DynamicPad(min_img_dim=(130, 70),
                                            padding_mode="constant", padding_value=125),
                                 transforms.RandomRotation(
                                     degrees=5, resample=PIL.Image.NEAREST, fill=125),
                                 transforms.RandomCrop(
                                     (60, 120), padding_mode="reflect"),
                                 transforms.Resize((40, 80)),
                                 transforms.RandomHorizontalFlip(p=0.5),
                                 transforms.RandomVerticalFlip(p=0.5),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.5], [0.5]),
                                 AddNoise(alpha=0.07)])


checkpoint_path = "experiments/munit_version_2_larger_image/lightning_logs/version_0/checkpoints/epoch={}.ckpt"
save_dir = "experiments/munit_version_2_larger_image/version_0/results"
datasetmodule = PostdamCarsDataModule(potsdam_dir,
                      data_dir2=artificial_dir,
                      transform=transform1,
                      transform2=transform2,
                      batch_size=1024)
datasetmodule.setup()
dataloader = datasetmodule.train_dataloader()

real, artificial = next(iter(dataloader))
styles = th.normal(0, 1, (1024, 8))

max_epochs = 1000
checkpoints = [i-1 for i in range(25, max_epochs, 25)] + [max_epochs-1]

numpy_file = os.path.join(save_dir, "fid_scores.npy")
if not os.path.exists(numpy_file):
    fid_train = []
    # main stuff
    with th.no_grad():
        for epoch in tqdm.tqdm(checkpoints, desc="Checkpoints"):
            path = checkpoint_path.format(epoch)
            if os.path.exists(path):
                # load model
                model = MUNIT.load_from_checkpoint(path)
                model.eval()
                model.cuda()


                encoder1 = model.generator.encoder1
                decoder2 = model.generator.decoder2

                contents, _ = encoder1(artificial.cuda())
                fake = decoder2(contents, styles.cuda())

                fid_train.append(fid_score(real, fake, device="cuda:0"))
                print(f"[FAKE] Epoch {epoch}: Train FID is {fid_train[-1]}.")
    np.save(numpy_file, {"epochs": checkpoints, "fid_train": fid_train})

# do the best epoch 5 times
data = np.load(numpy_file, allow_pickle=True).item()
fid_train = data.get("fid_train", [])
idx = np.argmin(fid_train)
epoch = checkpoints[idx]
path = checkpoint_path.format(epoch)

# load model
with th.no_grad():
    model = MUNIT.load_from_checkpoint(path)
    model.eval()
    model.cuda()
    encoder1 = model.generator.encoder1
    decoder2 = model.generator.decoder2
    
    fid_multiple = []
    for _ in range(5):
        real, artificial = next(iter(dataloader))
        styles = th.normal(0, 1, (1024, 8))
        contents, _ = encoder1(artificial.cuda())
        fake = decoder2(contents, styles.cuda())
        fid_multiple.append(fid_score(real, fake, device="cuda:0"))
    fid_multiple = np.array(fid_multiple)
    print(f"FID MIN - {fid_multiple.min()} | FID MAX - {fid_multiple.max()} | FID Mean - {fid_multiple.min()}")

# PLOT STUFF
plt.figure()
plt.plot(checkpoints[:len(fid_train)], fid_train, marker="*")
plt.xlabel("Epochs", fontsize=18)
plt.ylabel("FID Score", fontsize=18)
# plt.title("Training FID Score", fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "fid_score_train.pgf"))
# close everything, i don't know how important it is
plt.clf()
plt.close()

# PLOT STUFF
plt.figure()
plt.plot(checkpoints[:len(fid_train)], fid_train, marker="*")
plt.xlabel("Epochs", fontsize=18)
plt.ylabel("FID Score", fontsize=18)
# plt.title("Training FID Score", fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "fid_score_train.png"))
# close everything, i don't know how important it is
plt.clf()
plt.close()

FID MIN - 1.6502502217549044 | FID MAX - 1.7104738051166275 | FID Mean - 1.6502502217549044
